In [1]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [2]:
dataset = 'JW2015'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/mcp.M114.045468-1.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for JW2015 ...
Raw data loaded.


,>Site information,Protein-position ID,First Protein,Amino acid Position within protein,Gene names,>Expression data,Intensity cap1,Intensity cap2,Intensity cap3,Intensity pre1,...,Phospho (STY) Probabilities,Position in peptide,Number of Phospho (STY),Sequence window,PEP,Score,Score for localization,Charge,m/z,Mass error [ppm]
0,>Site information,E9PGG4:Y1022,E9PGG4,Y1022,ALS2CR11,>Expression data,386530.0,179460.0,155270.0,0.0,...,AEFIQEDQNMFPQDS(0.016)S(0.05)Y(0.179)Y(0.81)S(...,18,2,PQDSSYYSIANKE,0.000620,90.729,90.729,3,962.38528,-0.159260
1,>Site information,E9PGG4:Y53,E9PGG4,Y53,ALS2CR11,>Expression data,565600.0,625090.0,615700.0,0.0,...,GS(0.015)EAS(0.051)S(0.181)VPY(0.751)ALNQGT(0....,9,1,EASSVPYALNQGT,0.014998,61.167,60.764,3,690.99630,-0.209940
2,>Site information,H3BQ01:S5,H3BQ01,S5,NDRG4,>Expression data,227700.0,204040.0,240110.0,0.0,...,MWAS(0.5)T(0.5)DMQEITK,4,1,__MMWASTDMQEI,0.007185,69.721,69.721,2,760.81168,-0.081286
3,>Site information,H3BQ01:T6,H3BQ01,T6,NDRG4,>Expression data,227700.0,204040.0,240110.0,0.0,...,MWAS(0.5)T(0.5)DMQEITK,5,1,_MMWASTDMQEIT,0.007185,69.721,69.721,2,760.81168,-0.081286
4,>Site information,P27797:S189,P27797,S189,CALR,>Expression data,1282000.0,780030.0,625390.0,0.0,...,IDNS(0.998)QVESGS(0.002)LEDDWDFLPPKK,4,1,EVKIDNSQVESGS,0.000303,92.886,57.147,3,867.05817,0.179960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3298,>Site information,D3DP46:Y127,D3DP46,Y127,SPCS3,>Expression data,237600.0,150530.0,0.0,389660.0,...,Y(1)FFFDDGNGLK,1,1,KDMKTKYFFFDDG,0.019795,66.595,66.595,2,701.78939,0.148940
3299,>Site information,O00330:T369,O00330,T369,PDHX,>Expression data,0.0,0.0,0.0,0.0,...,QLPFIDIS(1)VAVAT(1)DK,13,2,ISVAVATDKGLLT,0.018220,71.513,71.513,2,888.91467,-0.383400
3300,>Site information,Q14749:S235,Q14749,S235,GNMT,>Expression data,0.0,0.0,0.0,0.0,...,AHMVT(0.001)LDYTVQVPGAGQDGS(0.997)PGLS(0.003)KFR,20,1,GAGQDGSPGLSKF,0.007568,59.943,59.943,3,976.46343,-0.020636
3301,>Site information,H7C0P1:S3,H7C0P1,S3,CCDC141,>Expression data,0.0,0.0,0.0,0.0,...,S(0.217)S(0.522)QGS(0.438)PS(0.229)VALS(0.229)...,2,4,____MSSQGSPSV,0.007557,35.215,35.215,3,1183.87750,0.765920


In [3]:
data['Phosphosite Probability'] = data['Phospho (STY) Probabilities'].str.findall(r'\(([\d\.]+)\)').apply(lambda x: list(map(float, x)))

# Take the maximum probability value from the list (you could choose min or average instead, but max is common)
data['Max Phosphosite Probability'] = data['Phosphosite Probability'].apply(lambda x: max(x) if x else 0)

# Now filter: keep rows where the max probability is >= 0.85
data = data[data['Max Phosphosite Probability'] >= 0.85]

# See what you have
print(data[['Phospho (STY) Probabilities', 'Max Phosphosite Probability']].head())


                         Phospho (STY) Probabilities  \
0  AEFIQEDQNMFPQDS(0.016)S(0.05)Y(0.179)Y(0.81)S(...   
4               IDNS(0.998)QVESGS(0.002)LEDDWDFLPPKK   
5              FAAFGESSS(0.003)S(0.078)GT(0.92)EMNNK   
7           RDGS(0.053)IQGPLS(0.928)AS(0.019)AIQEWCR   
9  S(0.888)VT(0.05)S(0.05)NQS(0.012)DGT(0.001)QES...   

   Max Phosphosite Probability  
0                        0.946  
4                        0.998  
5                        0.920  
7                        0.928  
9                        0.888  


In [4]:
data

,>Site information,Protein-position ID,First Protein,Amino acid Position within protein,Gene names,>Expression data,Intensity cap1,Intensity cap2,Intensity cap3,Intensity pre1,...,Number of Phospho (STY),Sequence window,PEP,Score,Score for localization,Charge,m/z,Mass error [ppm],Phosphosite Probability,Max Phosphosite Probability
0,>Site information,E9PGG4:Y1022,E9PGG4,Y1022,ALS2CR11,>Expression data,386530.0,179460.0,155270.0,0.0,...,2,PQDSSYYSIANKE,6.203200e-04,90.729,90.729,3,962.38528,-0.159260,"[0.016, 0.05, 0.179, 0.81, 0.946]",0.946
4,>Site information,P27797:S189,P27797,S189,CALR,>Expression data,1282000.0,780030.0,625390.0,0.0,...,1,EVKIDNSQVESGS,3.033130e-04,92.886,57.147,3,867.05817,0.179960,"[0.998, 0.002]",0.998
5,>Site information,P59282:T23,P59282,T23,TPPP2,>Expression data,1159500.0,832900.0,799460.0,0.0,...,1,ESSSSGTEMNNKN,1.130000e-65,217.480,216.000,2,922.36386,-0.171560,"[0.003, 0.078, 0.92]",0.920
7,>Site information,Q499Z3:S357,Q499Z3,S357,SLFNL1,>Expression data,711510.0,870330.0,477670.0,0.0,...,1,SIQGPLSASAIQE,1.177290e-03,95.662,88.551,3,737.67193,-0.085948,"[0.053, 0.928, 0.019]",0.928
9,>Site information,Q9UGV2:S346,Q9UGV2,S346,NDRG3,>Expression data,162340.0,80593.0,111900.0,0.0,...,1,ESPFSRSVTSNQS,3.794020e-03,62.739,62.739,3,831.00240,0.938460,"[0.888, 0.05, 0.05, 0.012, 0.001]",0.888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3297,>Site information,A6NEK1:S329,A6NEK1,S329,ARRDC5,>Expression data,554870.0,259330.0,219650.0,40867.0,...,1,SAICQLSEDGVLP,6.070680e-03,56.021,53.064,3,1189.91820,0.668050,"[0.001, 0.998]",0.998
3298,>Site information,D3DP46:Y127,D3DP46,Y127,SPCS3,>Expression data,237600.0,150530.0,0.0,389660.0,...,1,KDMKTKYFFFDDG,1.979490e-02,66.595,66.595,2,701.78939,0.148940,[1.0],1.000
3299,>Site information,O00330:T369,O00330,T369,PDHX,>Expression data,0.0,0.0,0.0,0.0,...,2,ISVAVATDKGLLT,1.821960e-02,71.513,71.513,2,888.91467,-0.383400,"[1.0, 1.0]",1.000
3300,>Site information,Q14749:S235,Q14749,S235,GNMT,>Expression data,0.0,0.0,0.0,0.0,...,1,GAGQDGSPGLSKF,7.567920e-03,59.943,59.943,3,976.46343,-0.020636,"[0.001, 0.997, 0.003]",0.997


In [5]:
data = data[~data['Amino acid Position within protein'].str.contains(';', na=False)]
data = data[~data['Gene names'].str.contains(';', na=False)]
data

,>Site information,Protein-position ID,First Protein,Amino acid Position within protein,Gene names,>Expression data,Intensity cap1,Intensity cap2,Intensity cap3,Intensity pre1,...,Number of Phospho (STY),Sequence window,PEP,Score,Score for localization,Charge,m/z,Mass error [ppm],Phosphosite Probability,Max Phosphosite Probability
0,>Site information,E9PGG4:Y1022,E9PGG4,Y1022,ALS2CR11,>Expression data,386530.0,179460.0,155270.0,0.0,...,2,PQDSSYYSIANKE,6.203200e-04,90.729,90.729,3,962.38528,-0.159260,"[0.016, 0.05, 0.179, 0.81, 0.946]",0.946
4,>Site information,P27797:S189,P27797,S189,CALR,>Expression data,1282000.0,780030.0,625390.0,0.0,...,1,EVKIDNSQVESGS,3.033130e-04,92.886,57.147,3,867.05817,0.179960,"[0.998, 0.002]",0.998
5,>Site information,P59282:T23,P59282,T23,TPPP2,>Expression data,1159500.0,832900.0,799460.0,0.0,...,1,ESSSSGTEMNNKN,1.130000e-65,217.480,216.000,2,922.36386,-0.171560,"[0.003, 0.078, 0.92]",0.920
7,>Site information,Q499Z3:S357,Q499Z3,S357,SLFNL1,>Expression data,711510.0,870330.0,477670.0,0.0,...,1,SIQGPLSASAIQE,1.177290e-03,95.662,88.551,3,737.67193,-0.085948,"[0.053, 0.928, 0.019]",0.928
9,>Site information,Q9UGV2:S346,Q9UGV2,S346,NDRG3,>Expression data,162340.0,80593.0,111900.0,0.0,...,1,ESPFSRSVTSNQS,3.794020e-03,62.739,62.739,3,831.00240,0.938460,"[0.888, 0.05, 0.05, 0.012, 0.001]",0.888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3297,>Site information,A6NEK1:S329,A6NEK1,S329,ARRDC5,>Expression data,554870.0,259330.0,219650.0,40867.0,...,1,SAICQLSEDGVLP,6.070680e-03,56.021,53.064,3,1189.91820,0.668050,"[0.001, 0.998]",0.998
3298,>Site information,D3DP46:Y127,D3DP46,Y127,SPCS3,>Expression data,237600.0,150530.0,0.0,389660.0,...,1,KDMKTKYFFFDDG,1.979490e-02,66.595,66.595,2,701.78939,0.148940,[1.0],1.000
3299,>Site information,O00330:T369,O00330,T369,PDHX,>Expression data,0.0,0.0,0.0,0.0,...,2,ISVAVATDKGLLT,1.821960e-02,71.513,71.513,2,888.91467,-0.383400,"[1.0, 1.0]",1.000
3300,>Site information,Q14749:S235,Q14749,S235,GNMT,>Expression data,0.0,0.0,0.0,0.0,...,1,GAGQDGSPGLSKF,7.567920e-03,59.943,59.943,3,976.46343,-0.020636,"[0.001, 0.997, 0.003]",0.997


In [6]:
data['Amino acid'] = data['Amino acid Position within protein'].str.extract(r'([A-Z])')
data['Positions within proteins'] = data['Amino acid Position within protein'].str.extract(r'(\d+)')

In [7]:
# filter data
data['Sequence window'] = data['Sequence window'].str.replace('_', '')
data

,>Site information,Protein-position ID,First Protein,Amino acid Position within protein,Gene names,>Expression data,Intensity cap1,Intensity cap2,Intensity cap3,Intensity pre1,...,PEP,Score,Score for localization,Charge,m/z,Mass error [ppm],Phosphosite Probability,Max Phosphosite Probability,Amino acid,Positions within proteins
0,>Site information,E9PGG4:Y1022,E9PGG4,Y1022,ALS2CR11,>Expression data,386530.0,179460.0,155270.0,0.0,...,6.203200e-04,90.729,90.729,3,962.38528,-0.159260,"[0.016, 0.05, 0.179, 0.81, 0.946]",0.946,Y,1022
4,>Site information,P27797:S189,P27797,S189,CALR,>Expression data,1282000.0,780030.0,625390.0,0.0,...,3.033130e-04,92.886,57.147,3,867.05817,0.179960,"[0.998, 0.002]",0.998,S,189
5,>Site information,P59282:T23,P59282,T23,TPPP2,>Expression data,1159500.0,832900.0,799460.0,0.0,...,1.130000e-65,217.480,216.000,2,922.36386,-0.171560,"[0.003, 0.078, 0.92]",0.920,T,23
7,>Site information,Q499Z3:S357,Q499Z3,S357,SLFNL1,>Expression data,711510.0,870330.0,477670.0,0.0,...,1.177290e-03,95.662,88.551,3,737.67193,-0.085948,"[0.053, 0.928, 0.019]",0.928,S,357
9,>Site information,Q9UGV2:S346,Q9UGV2,S346,NDRG3,>Expression data,162340.0,80593.0,111900.0,0.0,...,3.794020e-03,62.739,62.739,3,831.00240,0.938460,"[0.888, 0.05, 0.05, 0.012, 0.001]",0.888,S,346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3297,>Site information,A6NEK1:S329,A6NEK1,S329,ARRDC5,>Expression data,554870.0,259330.0,219650.0,40867.0,...,6.070680e-03,56.021,53.064,3,1189.91820,0.668050,"[0.001, 0.998]",0.998,S,329
3298,>Site information,D3DP46:Y127,D3DP46,Y127,SPCS3,>Expression data,237600.0,150530.0,0.0,389660.0,...,1.979490e-02,66.595,66.595,2,701.78939,0.148940,[1.0],1.000,Y,127
3299,>Site information,O00330:T369,O00330,T369,PDHX,>Expression data,0.0,0.0,0.0,0.0,...,1.821960e-02,71.513,71.513,2,888.91467,-0.383400,"[1.0, 1.0]",1.000,T,369
3300,>Site information,Q14749:S235,Q14749,S235,GNMT,>Expression data,0.0,0.0,0.0,0.0,...,7.567920e-03,59.943,59.943,3,976.46343,-0.020636,"[0.001, 0.997, 0.003]",0.997,S,235


In [8]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 
    

In [9]:
data = match_seq_to_genename(data, 'Sequence window')

PQDSSYYSIANKE
Match found for sequence: ID: sp|Q53TS8|CTSRT_HUMAN
Name: sp|Q53TS8|CTSRT_HUMAN
Description: sp|Q53TS8|CTSRT_HUMAN Cation channel sperm-associated targeting subunit tau OS=Homo sapiens OX=9606 GN=C2CD6 PE=1 SV=2
Number of features: 0
Seq('MEPPQETNRPFSTLDNRSGQVQVLSATPLLQRNPYSSPDIMHIKGSEASSVPYA...RGY')
Gene name match: <re.Match object; span=(100, 108), match='GN=C2CD6'>
Match found: PQDSSYYSIANKE -> C2CD6
EVKIDNSQVESGS
Match found for sequence: ID: sp|P27797|CALR_HUMAN
Name: sp|P27797|CALR_HUMAN
Description: sp|P27797|CALR_HUMAN Calreticulin OS=Homo sapiens OX=9606 GN=CALR PE=1 SV=1
Number of features: 0
Seq('MLLSVPLLLGLLGLAVAEPAVYFKEQFLDGDGWTSRWIESKHKSDFGKFVLSSG...DEL')
Gene name match: <re.Match object; span=(58, 65), match='GN=CALR'>
Match found: EVKIDNSQVESGS -> CALR
ESSSSGTEMNNKN
Match found for sequence: ID: sp|P59282|TPPP2_HUMAN
Name: sp|P59282|TPPP2_HUMAN
Description: sp|P59282|TPPP2_HUMAN Tubulin polymerization-promoting protein family member 2 OS=Homo sapiens OX=

In [10]:
data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Positions within proteins'].astype(str) + ')'

In [11]:
keepcols = ['Phosphosite'] + ['GeneName'] + [col for col in data.columns if 'Intensity' in col]
data = data[keepcols]
data

,Phosphosite,GeneName,Intensity cap1,Intensity cap2,Intensity cap3,Intensity pre1,Intensity pre2,Intensity pre3
0,Y(1022),C2CD6,386530.0,179460.0,155270.0,0.0,0.0,0.0
4,S(189),CALR,1282000.0,780030.0,625390.0,0.0,0.0,0.0
5,T(23),TPPP2,1159500.0,832900.0,799460.0,0.0,0.0,0.0
7,S(357),SLFNL1,711510.0,870330.0,477670.0,0.0,0.0,0.0
9,S(346),NDRG3,162340.0,80593.0,111900.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
3297,S(329),ARRDC5,554870.0,259330.0,219650.0,40867.0,1324100.0,52075.0
3298,Y(127),SPCS3,237600.0,150530.0,0.0,389660.0,600670.0,102420.0
3299,T(369),PDHX,0.0,0.0,0.0,0.0,0.0,535300.0
3300,S(235),GNMT,0.0,0.0,0.0,0.0,0.0,706790.0


In [12]:
Intensity_columns = [col for col in data.columns if 'Intensity' in col]
data[Intensity_columns] = data[Intensity_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_11699/3047076211.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[Intensity_columns] = data[Intensity_columns].apply(pd.to_numeric, errors='coerce')


In [13]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.Dataframe>
    
    out:
    ====
    dataset: <pd.Dataframe> with log2 transformed values

    '''
    cols_to_transform = dataset[Intensity_columns]
    dataset[Intensity_columns] = cols_to_transform.apply(np.log2)
    print('Data has been log2 transformed.')
    return dataset


In [14]:
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}") # Print the DataFrame after log2 transformation
data

Data has been log2 transformed.
DataFrame after log2 transformation:
     Phosphosite GeneName  Intensity cap1  Intensity cap2  Intensity cap3  \
0        Y(1022)    C2CD6       18.560221       17.453303       17.244420   
4         S(189)     CALR       20.289965       19.573170       19.254397   
5          T(23)    TPPP2       20.145071       19.667784       19.608666   
7         S(357)   SLFNL1       19.440525       19.731203       18.865655   
9         S(346)    NDRG3       17.308659       16.298367       16.771851   
...          ...      ...             ...             ...             ...   
3297      S(329)   ARRDC5       19.081790       17.984430       17.744847   
3298      Y(127)    SPCS3       17.858175       17.199692            -inf   
3299      T(369)     PDHX            -inf            -inf            -inf   
3300      S(235)     GNMT            -inf            -inf            -inf   
3301        S(3)  CCDC141            -inf            -inf            -inf   

      

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_11699/2333207401.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[Intensity_columns] = cols_to_transform.apply(np.log2)


,Phosphosite,GeneName,Intensity cap1,Intensity cap2,Intensity cap3,Intensity pre1,Intensity pre2,Intensity pre3
0,Y(1022),C2CD6,18.560221,17.453303,17.244420,-inf,-inf,-inf
4,S(189),CALR,20.289965,19.573170,19.254397,-inf,-inf,-inf
5,T(23),TPPP2,20.145071,19.667784,19.608666,-inf,-inf,-inf
7,S(357),SLFNL1,19.440525,19.731203,18.865655,-inf,-inf,-inf
9,S(346),NDRG3,17.308659,16.298367,16.771851,-inf,-inf,-inf
...,...,...,...,...,...,...,...,...
3297,S(329),ARRDC5,19.081790,17.984430,17.744847,15.318649,20.336581,15.668303
3298,Y(127),SPCS3,17.858175,17.199692,-inf,18.571856,19.196213,16.644138
3299,T(369),PDHX,-inf,-inf,-inf,-inf,-inf,19.029988
3300,S(235),GNMT,-inf,-inf,-inf,-inf,-inf,19.430922


In [15]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.' )

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_11699/1104027526.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [16]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains(';', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains('-', case=False, na=False)]

    # Add this line to remove decimals from phosphosite_ID (e.g., S123.0 -> S123)
    data['phosphosite_ID'] = data['phosphosite_ID'].apply(lambda x: re.sub(r'\((\d+)\.0+\)', r'(\1)', x))
    
    data_grouped = data.groupby(by='phosphosite_ID')
    
    if len(data) != len(data_grouped):
        numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
        non_numeric_cols = data.columns.difference(numeric_cols + ['phosphosite_ID']).tolist()
        data_numeric = data_grouped[numeric_cols].mean()
        data_categorical = data_grouped[non_numeric_cols].first().reset_index()
        
        # Merge numeric and non-numeric parts
        data = pd.merge(data_categorical, data_numeric, on='phosphosite_ID')
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')

    # Replace inf values with NaNs
    data = data.replace([np.inf, -np.inf], np.nan)
    
    # Ensure phosphosite_ID is first column
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)

    return data

In [17]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
After cleaning phosphosite_ID column:


,phosphosite_ID,Intensity cap1,Intensity cap2,Intensity cap3,Intensity pre1,Intensity pre2,Intensity pre3
0,AARSD1_S(137),22.462388,21.852776,22.355254,22.330658,23.032882,21.230813
1,AARSD1_S(145),19.050699,18.524549,18.530162,16.045610,16.762019,16.280590
2,AARSD1_S(174),22.354125,22.377242,22.327289,22.006348,22.699139,22.707378
3,AARSD1_S(184),21.618002,21.621226,21.254074,20.896965,21.623460,21.460190
4,AARSD1_S(206),24.488734,23.446196,23.361318,23.435172,23.707778,23.492650
...,...,...,...,...,...,...,...
2187,ZDHHC11_S(10),19.313228,18.736514,18.336690,19.570671,18.848061,18.647796
2188,ZDHHC11_S(5),NaN,18.736514,18.264534,18.996948,NaN,18.165457
2189,ZDHHC11_S(7),18.962794,18.736514,13.979515,19.570671,18.848061,18.647796
2190,ZMYND10_S(364),17.745635,16.643152,17.637927,NaN,17.014979,16.803601


In [18]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/JW2015.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

JW2015 has been saved to CSV successfully!       phosphosite_ID  Intensity cap1  Intensity cap2  Intensity cap3  \
0      AARSD1_S(137)       22.462388       21.852776       22.355254   
1      AARSD1_S(145)       19.050699       18.524549       18.530162   
2      AARSD1_S(174)       22.354125       22.377242       22.327289   
3      AARSD1_S(184)       21.618002       21.621226       21.254074   
4      AARSD1_S(206)       24.488734       23.446196       23.361318   
...              ...             ...             ...             ...   
2187   ZDHHC11_S(10)       19.313228       18.736514       18.336690   
2188    ZDHHC11_S(5)             NaN       18.736514       18.264534   
2189    ZDHHC11_S(7)       18.962794       18.736514       13.979515   
2190  ZMYND10_S(364)       17.745635       16.643152       17.637927   
2191    ZNRD2_S(103)       22.052717       22.006382       21.407639   

      Intensity pre1  Intensity pre2  Intensity pre3  
0          22.330658       23.032882 